In [1]:
#papermill parameters
base_directory = "$(homedir())/workspace/sars-cov2-pangenome-analysis/"
sequences_directory = "$(base_directory)/data/sequences"
metadata_file = "$(base_directory)/metadata/sequences.csv"

"/home/jovyan/workspace/sars-cov2-pangenome-analysis//metadata/sequences.csv"

In [ ]:
import Pkg

pkgs = [
    "Revise",
    "MetaGraphs",
    "Graphs",
    "JSON",
    "uCSV",
    "DataFrames"
]

Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

# Pkg.add(url="https://github.com/cjprybol/Mycelia.git", rev="master")
# Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia

    Updating registry at `/opt/julia/registries/General`
   Resolving package versions...
    Updating `~/work/Mycelia/Project.toml`
  [295af30f] + Revise v3.4.0
    Updating `~/work/Mycelia/Manifest.toml`
  [da1fd8a2] + CodeTracking v1.1.0
  [aa1ae85d] + JuliaInterpreter v0.9.15
  [6f1432cf] + LoweredCodeUtils v2.2.2
  [295af30f] + Revise v3.4.0
  [7b1f6079] + FileWatching
Precompiling project...
  ✗ Mycelia
  0 dependencies successfully precompiled in 7 seconds (228 already precompiled)
  1 dependency errored. To see a full report either run `import Pkg; Pkg.precompile()` or load the package
┌ Info: Precompiling Mycelia [453d265d-8292-4a7b-a57c-dce3f9ae6acd]
└ @ Base loading.jl:1342


In [ ]:
sequence_metadata = DataFrames.DataFrame(uCSV.read(metadata_file, quotes='"', header=1, typedetectrows=100)...)

In [ ]:
run(`head $(metadata_file)`)

In [ ]:
readdir(sequences_directory, join=true)
# fastx_files = filter(x -> occursin("genomic.fna", x), 

In [ ]:
# import BioSequences
# import FASTX
# import Kmers

In [ ]:
graph = Mycelia.initialize_graph()
graph = Mycelia.add_fastx_to_graph!(graph, fastx_files)

In [ ]:
# add fastx metadata to graphs

In [ ]:
data_report_json_file = joinpath(data_directory, "data_report.jsonl")
data_report_json = JSON.parse(open(data_report_json_file))
identifier_to_data_report = Dict(data_report_json["accession"] => data_report_json)
Mycelia.add_metadata_to_graph!(graph, identifier_to_data_report)

In [ ]:
# for (key, value) in graph.vprops[1]
#     println(key)
#     println(value)
#     println()
# end

In [ ]:
function add_biosample_to_graph!(graph, biosample_json)
    try
        graph[biosample_json["identifier"], :identifier]
        @info "node $(biosample_json["identifier"]) already present"
    catch
        Graphs.add_vertex!(graph)
        vertex_id = Graphs.nv(graph)

        MetaGraphs.set_prop!(graph, vertex_id, :TYPE, "BIOSAMPLE")
        for (key, value) in biosample_json
            MetaGraphs.set_prop!(graph, vertex_id, Symbol(key), value)
        end
    end
    return graph
end

function add_biosamples_to_graph!(graph, biosample_json_file)
    open(biosample_json_file) do io
        # for i in 1:1
            # line = readline(io)
        for line in eachline(io)
            jsonl = JSON.parse(line)
            jsonl["attributes"] = Dict(attribute["name"] => attribute["value"] for attribute in jsonl["attributes"])
            jsonl["identifier"] = jsonl["accession"]
            add_biosample_to_graph!(graph, jsonl)
        end
    end
end

In [ ]:
biosample_json_file = joinpath(data_directory, "biosample.jsonl")
add_biosamples_to_graph!(graph, biosample_json_file)
# @time biosample_json = JSON.parse(open(biosample_json_file))

In [ ]:
# kmer_size = Mycelia.assess_dnamer_saturation(fastx_files)